# IDA 2016 Correct Missing Value Coding

In [ ]:
import sys
sys.path.append("..")
from pyspark.sql.functions import col
from pyspark.sql.session import SparkSession
from helpers.path_translation import translate_to_file_string
from helpers.data_prep_and_print import print_df

In [ ]:
input_file = translate_to_file_string("../data/ida_2016_training_set_update.csv")

In [ ]:
spark = (SparkSession
       .builder   
       .master("local[*]")
       .appName("Ida2016 Data Cleaning")
       .getOrCreate())

## Read in the file and replace na with None

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(input_file) 
print(df.na.drop().count())
df_with_corr_null = df.replace(to_replace='na', value=None)
df_with_corr_null.summary().show()

## Calculation of Null Values per Column

In [ ]:
print(df_with_corr_null.count())
Dict_Null = {col:df_with_corr_null.filter(df_with_corr_null[col].isNull()).count() for col in df_with_corr_null.columns}
print(Dict_Null)

## Write the result to a new csv file

In [ ]:
output_file = "../data/ida_2016_cleaned.csv"
# df_with_corr_null.write.option("header", "true").csv(output_file)
df_with_corr_null.toPandas().to_csv(output_file,index=False) # Save the data into a single file to a permanent location

In [ ]:
spark.stop()